In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.metrics import r2_score
import joblib
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt



n_particles = 20
max_iter = 30
error_threshold = 0.001
w = 0.9
c1 = 2.0
c2 = 1.0


param_bounds = [(4, 13)] * 4 + [(4, 13)] + [(4, 13)] * 2 + [(16, 112)] * 2

param_names = ['top','left','bottom','right','diag1','diag2','vert','hori','x','y']

model_A = load_model('best_model_A.h5')
model_B = load_model('best_model_B.h5', compile=False)
scaler_X = joblib.load('scaler_X.pkl')
scaler_Y = joblib.load('scaler_Y.pkl')
scaler_curve = joblib.load('ss.pkl')


In [ ]:
def expand_to_full_params(params_9d):
    full = np.zeros(10, dtype=int)
    full[0:4] = params_9d[0:4]
    full[4] = full[5] = params_9d[4]  # diag2 = diag1
    full[6:10] = params_9d[5:9]
    return full

def generate_image_from_params(params):
    top, left, bottom, right, diag1, diag2, vert, hori, x, y = params.astype(int)
    img = np.ones((128, 128), dtype=np.uint8) * 255
    cv2.rectangle(img, (0, 0), (127, top - 1), 0, -1)
    cv2.rectangle(img, (0, 0), (left - 1, 127), 0, -1)
    cv2.rectangle(img, (0, 128 - bottom), (127, 127), 0, -1)
    cv2.rectangle(img, (128 - right, 0), (127, 127), 0, -1)
    cv2.line(img, (0, 0), (127, 127), 0, diag1)
    cv2.line(img, (0, 127), (127, 0), 0, diag2)
    cv2.line(img, (x, 0), (x, 127), 0, vert)
    cv2.line(img, (0, y), (127, y), 0, hori)
    top_half = np.hstack([img, np.fliplr(img)])
    full_img = np.vstack([top_half, np.flipud(top_half)])
    return full_img

def calculate_porosity(img):
    return np.sum(img == 255) / img.size

def predict_curve_from_image(img_rgb, phi):
    img_input = img_rgb[np.newaxis, ...] / 255.0
    phi_input = np.array([[phi]], dtype=np.float32)
    pred_std = model_A.predict([img_input, phi_input], verbose=0)[0].reshape(1, -1)
    return scaler_curve.inverse_transform(pred_std)[0]

def compute_error(true_curve, pred_curve):
    return 1 - r2_score(true_curve, pred_curve)

os.makedirs('final_generated_images', exist_ok=True)
all_params, all_curves, all_errors = [], [], []

In [ ]:
import time

total_start_time = time.time()
time_records = []   


df_all = pd.read_excel('target_curve.xlsx')
x_vals = df_all.iloc[1:, 0].values
curve_names = df_all.columns[1:]

os.makedirs('comparison_curves', exist_ok=True)
os.makedirs('error_curves', exist_ok=True)


for idx, name in enumerate(curve_names):
    curve_start_time = time.time()   
    print(f"\n🔍 opt：{name}")
    y_target = df_all[name].iloc[1:].values.astype(float)
    norm_target = scaler_X.transform(y_target.reshape(1, -1)).reshape(1, 200, 1)

    class Particle:
        def __init__(self, init_pos=None):
            self.position = np.array(init_pos, dtype=np.float64) if init_pos is not None else self.random_position()
            self.velocity = np.random.uniform(-1, 1, size=9)
            self.best_position = self.position.copy()
            self.best_error = float('inf')
            self.error = float('inf')
        def random_position(self):
            return np.array([np.random.randint(low, high+1) for low, high in param_bounds], dtype=np.float64)
        def evaluate(self, target_curve):
            full_param = expand_to_full_params(np.round(self.position).astype(int))
            img = generate_image_from_params(full_param)
            phi = calculate_porosity(img)
            img_rgb = cv2.cvtColor(cv2.merge([img]*3), cv2.COLOR_BGR2RGB)
            pred_curve = predict_curve_from_image(img_rgb, phi)
            self.error = compute_error(target_curve, pred_curve)
            if self.error < self.best_error:
                self.best_position = self.position.copy()
                self.best_error = self.error
        def update_velocity(self, global_best):
            r1, r2 = np.random.rand(2)
            cognitive = c1 * r1 * (self.best_position - self.position)
            social = c2 * r2 * (global_best - self.position)
            self.velocity = w * self.velocity + cognitive + social
        def update_position(self):
            self.position += self.velocity
            for i, (low, high) in enumerate(param_bounds):
                self.position[i] = np.clip(int(round(self.position[i])), low, high)


    pred_init = model_B.predict(norm_target)
    params_init = scaler_Y.inverse_transform(pred_init)[0]
    params_init = np.clip(np.round(params_init), 1, 128).astype(int)
    params_init_short = np.delete(params_init, 5)

    swarm = [Particle(init_pos=params_init_short)] + [Particle() for _ in range(n_particles - 1)]
    global_best_position = None
    global_best_error = float('inf')
    error_log = []


    for it in range(max_iter):
        for p in swarm:
            p.evaluate(y_target)
            if p.error < global_best_error:
                global_best_error = p.error
                global_best_position = p.position.copy()
        error_log.append(global_best_error)
        if global_best_error <= error_threshold:
            break
        for p in swarm:
            p.update_velocity(global_best_position)
            p.update_position()


    best_param_full = expand_to_full_params(global_best_position)
    img = generate_image_from_params(best_param_full)
    img_rgb = cv2.cvtColor(cv2.merge([img]*3), cv2.COLOR_BGR2RGB)
    phi = calculate_porosity(img)
    img_name = name if name.endswith('.jpg') else name + '.jpg'
    cv2.imwrite(f'final_generated_images/{img_name}', img_rgb)


    pred_curve = predict_curve_from_image(img_rgb, phi)
    all_curves.append(pd.Series(pred_curve, name=name))
    all_params.append({'name': name, **{k: int(v) for k, v in zip(param_names, best_param_full)}})
    all_errors.append(pd.Series(error_log, name=name))
    

    curve_end_time = time.time()
    curve_elapsed = curve_end_time - curve_start_time
    print(f"⏱ curve {name} time：{curve_elapsed:.2f} s（about {curve_elapsed/60:.2f} min）")

 
    time_records.append({
        "name": str(name),
        "seconds": float(curve_elapsed),
        "minutes": float(curve_elapsed/60.0),
        "iterations": len(error_log) if 'error_log' in locals() else None,
        "best_error_(1-R2)": float(global_best_error) if 'global_best_error' in locals() else None
    })


    plt.figure(figsize=(6, 4))
    plt.plot(x_vals, y_target, label='Target')
    plt.plot(x_vals, pred_curve, '--', label='Predicted')
    plt.title(f'{name} Curve'); plt.legend(); plt.grid()
    plt.tight_layout()
    plt.savefig(f'comparison_curves/compare_{name}.png')
    if idx < 2:
        plt.show()
    plt.close()
    
    plt.figure()
    plt.plot(error_log, marker='o')
    plt.title(f'{name} Error Convergence')
    plt.xlabel('Iteration'); plt.ylabel('1 - R²')
    plt.tight_layout()
    plt.savefig(f'error_curves/error_{name}.png')
    if idx < 2:
        plt.show()
    plt.close()
    
     


pd.DataFrame(all_params)[['name'] + param_names].to_excel('final_best_parameters.xlsx', index=False)


df_all_curves = pd.concat(all_curves, axis=1)
df_all_curves.insert(0, 'Displacement', x_vals)
df_all_curves.to_excel('final_best_predicted_curves.xlsx', index=False)


df_all_errors = pd.concat(all_errors, axis=1)
df_all_errors.insert(0, 'Iteration', list(range(1, len(all_errors[0]) + 1)))
df_all_errors.to_excel('error_per_iteration.xlsx', index=False)


total_end_time = time.time()
total_elapsed = total_end_time - total_start_time
print("ok！")
print(f"time：{total_elapsed:.2f} s（about {total_elapsed/60:.2f} min）")


report_path = 'pso_time_report.xlsx'
df_times = pd.DataFrame(time_records)


total_row = {
    "name": "TOTAL",
    "seconds": total_elapsed,
    "minutes": total_elapsed/60.0,
    "iterations": None,
    "best_error_(1-R2)": None
}
df_times_with_total = pd.concat([df_times, pd.DataFrame([total_row])], ignore_index=True)


summary = pd.DataFrame({
    "metric": [
        "num_curves",
        "total_seconds",
        "total_minutes",
        "avg_seconds_per_curve",
        "avg_minutes_per_curve"
    ],
    "value": [
        len(df_times),
        total_elapsed,
        total_elapsed/60.0,
        (total_elapsed/len(df_times)) if len(df_times) > 0 else None,
        (total_elapsed/60.0/len(df_times)) if len(df_times) > 0 else None
    ]
})

with pd.ExcelWriter(report_path, engine='xlsxwriter') as writer:
    df_times_with_total.to_excel(writer, index=False, sheet_name='PerCurve')
    summary.to_excel(writer, index=False, sheet_name='Summary')

print(f"save{report_path}")



print("ok")